<a href="https://colab.research.google.com/github/jyotidabass/Chatbot/blob/main/SQLite_and_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Here's a simple Python code to implement the chatbot on Google Colab with SQLite database and Pinecone embeddings. We'll use SQLite for storing the candidate data, and Pinecone for handling the semantic part of the query.**

First, we'll set up the SQLite database and create the necessary tables. Then, we'll populate the database with some sample candidates. After that, we'll use Pinecone to handle the natural language queries, and we'll create a simple front-end to interact with the chatbot.

Set up SQLite database and create tables:
Create a SQLite database and create the necessary tables:

In [56]:
import sqlite3

# create a database
conn = sqlite3.connect('candidates.db')
c = conn.cursor()

# create the candidates table
c.execute("""CREATE TABLE IF NOT EXISTS candidates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    title TEXT,
    company TEXT,
    skills TEXT,
    full_time TEXT,
    budget INTEGER,
    github_username TEXT
)""")

# create the embeddings table
c.execute("""CREATE TABLE IF NOT EXISTS embeddings (
    id INTEGER PRIMARY KEY,
    candidate_id INTEGER,
    embedding TEXT
)""")

# create indexes for candidate_id
c.execute("CREATE INDEX IF NOT EXISTS candidate_idx ON embeddings (candidate_id)")

# commit the changes
conn.commit()

**Populate the SQLite database with sample candidates:**

Populate the database with a few sample candidates. In this example, we'll use a CSV file sample_candidates.csv for simplicity, but you can populate the database with your data in any way you prefer.

In [57]:
import sqlite3

# create a database
conn = sqlite3.connect('candidates.db')
c = conn.cursor()

# recreate the candidates table with the 'title' column
c.execute("DROP TABLE IF EXISTS candidates") # Drop the existing table if it exists
c.execute("""CREATE TABLE candidates (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    title TEXT,
    company TEXT,
    skills TEXT,
    full_time TEXT,
    budget INTEGER,
    github_username TEXT
)""") # SQL comments are done with '--' instead of '#'

# create the embeddings table
c.execute("""CREATE TABLE IF NOT EXISTS embeddings (
    id INTEGER PRIMARY KEY,
    candidate_id INTEGER,
    embedding TEXT
)""")

# create indexes for candidate_id
c.execute("CREATE INDEX IF NOT EXISTS candidate_idx ON embeddings (candidate_id)")

# commit the changes
conn.commit()

**Set up Pinecone embeddings:**

First, you need to create an API key and a workspace in Pinecone.

Once you have your API key and workspace, install the Pinecone Python package and create a Pinecone client:

In [58]:
!pip install pinecone

from pinecone import Pinecone

# Pinecone API key
API_KEY = 'YOUR_API_KEY'

# Pinecone workspace
WORKSPACE = 'YOUR_WORKSPACE'

# create a Pinecone client
client = Pinecone(API_KEY, workspace=WORKSPACE)

# **Implement the chatbot:**

Now that we have our database and Pinecone client set up, we can implement the chatbot. We'll use SQLAlchemy to interact with the SQLite database.

In [59]:
from sqlalchemy import create_engine, func

# SQL Alchemy engine
engine = create_engine('sqlite:///candidates.db')

# create a connection
connection = engine.connect()

# **Create a simple front-end for the chatbot:**

We'll create a simple front-end using Python's input function to interact with the chatbot. You can replace this with a more advanced front-end if needed.

In [60]:
# front-end
def chatbot_loop():
    while True:
        query = input("Enter your query: ")

        # handle the query
        results = search_candidates(query)

        # show the results
        if results:
            print("Here are some candidates that match your query:")
            for r in results:
                print(f"Name: {r['name']}, Title: {r['title']}, Company: {r['company']}")
        else:
            print("No candidates found.")

        # ask for follow-up query
        follow_up = input("Do you want a follow-up query (y/n)? ")
        if follow_up.lower() == 'n':
            break

# **Implement the search function:**

We'll create a search function that takes the user's query and returns the best candidates. This function will handle the scalar part of the query using SQLAlchemy and the semantic part using Pinecone embeddings.

In [61]:
# search function
def search_candidates(query):
    # handle scalar queries
    if 'full-time' in query.lower():
        candidates = connection.execute(
            "SELECT * FROM candidates WHERE full_time = 'Full Time' AND budget >=?  ORDER BY budget DESC",
            [float(input("Enter budget: "))])
    else:
        candidates = connection.execute("SELECT * FROM candidates ORDER BY relevance DESC")

    # handle semantic search
    vector = client.search(query, model='general/doc2vec-cbow-300', limit=3)['results']
    candidates_semantic = [c[0] for c in vector]

    # return the best candidates
    candidates = [dict(row) for row in candidates]
    return candidates[:3] if set(candidates_semantic).intersection(candidates) else []

# **Run the chatbot:**

Finally, run the chatbot loop.

In [ ]:
if __name__ == '__main__':
    chatbot_loop()

Enter your query: full-time


**This code will create a simple chatbot that interacts with the candidates' data stored in the SQLite database. **

The natural language queries are handled by Pinecone using embeddings. When you run the script, you'll be prompted to enter a query, and the chatbot will return the best candidates based on your input.